In [ ]:
#| default_exp pds.apps

# Apps
> User interface apps/functions to make using the PDS tools easy.

In [ ]:
#| hide
from nbdev.showdoc import show_doc  # noqa

In [ ]:
#| export
import pandas as pd
from planetarypy.config import config
from planetarypy.pds.indexes import Index

In [ ]:
#| export
def find_indexes(
        instrument: str,  # Dotted mission.instrument key, e.g. cassini.iss
) -> list:  # List of configured index names
    "Find existing indexes for an instrument."
    return config.list_indexes(instrument)

In [ ]:
find_indexes("cassini.iss")

['index', 'inventory', 'moon_summary', 'ring_summary', 'saturn_summary']

In [ ]:
find_indexes("mro.ctx")

['edr']

In [ ]:
#| export
def get_index(
        instr: str,  # Dotted instrument index, e.g. cassini.iss
        index_name: str = '',  # Index name, for exmample 'moon_summary. Optional'
        # switch to refresh an index (i.e. download if update available).
        # Set to False for faster return time to avoid web scraping
        refresh: bool = True,  
        force: bool = False,  # switch off for faster return time.
) -> pd.DataFrame:  # The PDS index convert to pandas DataFrame
    """Example: get_index("cassini.iss", "index")"""
    # I need to add the check_update switch to the constructor b/c of dynamic url setting that always
    # wants to go online to find the latest volume URL.
    if not index_name:
        index = Index(instr, check_update=refresh)
    else:
        index = Index(instr + ".indexes." + index_name, check_update=refresh)
    if not index.local_table_path.exists() or force:
        index.download()
    elif refresh and index.update_available:
        index.download()
        print("An updated index is available. Downloading...")
    if not index.local_parq_path.exists():
        index.convert_to_parquet()
    return index.parquet

In [ ]:
instr = "cassini.iss"
index_name = "moon_summary"

In [ ]:
index = get_index(instr, index_name, refresh=False)

In [ ]:
index

,VOLUME_ID,FILE_SPECIFICATION_NAME,OPUS_ID,TARGET_NAME,MINIMUM_PLANETOCENTRIC_LATITUDE,MAXIMUM_PLANETOCENTRIC_LATITUDE,MINIMUM_PLANETOGRAPHIC_LATITUDE,MAXIMUM_PLANETOGRAPHIC_LATITUDE,MINIMUM_IAU_LONGITUDE,MAXIMUM_IAU_LONGITUDE,...,MAXIMUM_EMISSION_ANGLE,SUB_SOLAR_PLANETOCENTRIC_LATITUDE,SUB_SOLAR_PLANETOGRAPHIC_LATITUDE,SUB_OBSERVER_PLANETOCENTRIC_LATITUDE,SUB_OBSERVER_PLANETOGRAPHIC_LATITUDE,SUB_SOLAR_IAU_LONGITUDE,SUB_OBSERVER_IAU_LONGITUDE,CENTER_RESOLUTION,CENTER_DISTANCE,CENTER_PHASE_ANGLE
0,COISS_2001,data/1454725799_1455008789/N1454725799_1.LBL,co-iss-n1454725799,RHEA,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-25.206,-25.442,-16.566,-16.558,183.907,252.947,423.6356,70701354.366,64.429
1,COISS_2001,data/1454725799_1455008789/N1454725799_1.LBL,co-iss-n1454725799,HELENE,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-25.532,-50.716,-16.532,-36.57,148.17,217.149,424.36902,70823757.002,64.345
2,COISS_2001,data/1454725799_1455008789/N1454725799_1.LBL,co-iss-n1454725799,TELESTO,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-26.061,-51.51,-15.639,-21.616,189.199,257.807,424.28746,70810144.519,64.28
3,COISS_2001,data/1454725799_1455008789/N1454726579_1.LBL,co-iss-n1454726579,TITAN,-36.111,-36.111,-36.111,-36.111,0.0,360.0,...,54.704,-25.325,-25.325,-16.28,-16.28,340.092,47.855,428.89673,71579395.066,63.37
4,COISS_2001,data/1454725799_1455008789/N1454727359_1.LBL,co-iss-n1454727359,HYPERION,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,11.777,25.907,44.104,57.544,173.985,237.769,418.94545,69918606.586,63.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385714,COISS_2116,data/1884017637_1884113741/N1884017911_1.LBL,co-iss-n1884017911,TITAN,-14.352,72.101,-14.352,72.101,229.956,329.975,...,89.069,26.644,26.644,23.599,23.599,32.084,299.389,4.64069,774493.2,81.887
385715,COISS_2116,data/1884017637_1884113741/N1884018021_1.LBL,co-iss-n1884018021,TITAN,-14.597,71.882,-14.597,71.882,230.932,330.255,...,88.913,26.644,26.644,23.594,23.594,32.107,299.415,4.64263,774816.731,81.888
385716,COISS_2116,data/1884017637_1884113741/N1884018579_1.LBL,co-iss-n1884018579,TITAN,-14.435,72.643,-14.435,72.643,228.942,330.644,...,88.869,26.644,26.644,23.57,23.57,32.217,299.536,9.30328,776321.466,81.889
385717,COISS_2116,data/1884017637_1884113741/N1884091792_1.LBL,co-iss-n1884091792,DAPHNIS,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,26.684,39.966,25.321,38.64,294.939,124.641,4.56619,762060.157,127.157


Indexes can be given as 2 parameters ("cassini.iss", "ring_summary") or one ("cassini.iss.ring_summary"):

In [ ]:
index = get_index("cassini.iss", "ring_summary", refresh=False)
index.head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,OPUS_ID,MINIMUM_RIGHT_ASCENSION,MAXIMUM_RIGHT_ASCENSION,MINIMUM_DECLINATION,MAXIMUM_DECLINATION,MINIMUM_RING_RADIUS,MAXIMUM_RING_RADIUS,FINEST_RING_INTERCEPT_RESOLUTION,...,RING_CENTER_DISTANCE,SUB_SOLAR_RING_LONGITUDE,SUB_OBSERVER_RING_LONGITUDE,RING_CENTER_PHASE_ANGLE,RING_CENTER_INCIDENCE_ANGLE,RING_CENTER_NORTH_BASED_INCIDENCE_ANGLE,RING_CENTER_EMISSION_ANGLE,RING_CENTER_NORTH_BASED_EMISSION_ANGLE,SOLAR_RING_OPENING_ANGLE,OBSERVER_RING_OPENING_ANGLE
0,COISS_2001,data/1454725799_1455008789/N1454725799_1.LBL,co-iss-n1454725799,35.528398,35.905555,9.719637,10.091212,205899.335,1098621.586,419.0137,...,70874998.947,284.523,215.951,64.07,64.589,115.411,73.676,106.324,-25.411,-16.324
1,COISS_2001,data/1454725799_1455008789/N1454726579_1.LBL,co-iss-n1454726579,36.662898,37.039038,9.590114,9.960833,721136.155,1745689.307,421.97145,...,70870851.371,284.524,215.951,64.07,64.589,115.411,73.676,106.324,-25.411,-16.324
2,COISS_2001,data/1454725799_1455008789/N1454727359_1.LBL,co-iss-n1454727359,36.928066,37.304334,9.879593,10.25011,1042341.438,2186830.92,414.79041,...,70866703.094,284.524,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324
3,COISS_2001,data/1454725799_1455008789/N1454728139_1.LBL,co-iss-n1454728139,28.10042,28.479618,6.224709,6.601574,-999.0,-999.0,-999.0,...,70862557.207,284.524,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324
4,COISS_2001,data/1454725799_1455008789/N1454728919_1.LBL,co-iss-n1454728919,37.981418,38.357791,10.611947,10.981697,3865717.589,3905151.343,405.91234,...,70858406.794,284.525,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324


In [ ]:
get_index("mro.ctx.edr", refresh=False)

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,...,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER
0,MROX_0001,DATA/CRU_000001_9999_XN_99N999W.IMG,4A_04_0001000400,CRU_000001_9999_XN_99N999W,2005-08-30 15:40:21.549,CTX,NIFL,5056,1024,1,...,0.0,0.0,0.0,0.0,278.89,10.16,0.0,Instrument checkout image of space,OK,-4242
1,MROX_0001,DATA/CRU_000002_9999_XN_99N999W.IMG,4A_04_0001000500,CRU_000002_9999_XN_99N999W,2005-09-08 15:59:45.313,CTX,NIFL,5056,15360,1,...,0.0,0.0,0.0,0.0,284.48,4.6,0.0,Calibration image of the Moon,OK,-4126
2,MROX_0001,DATA/CRU_000003_9999_XN_99N999W.IMG,4A_04_0001000600,CRU_000003_9999_XN_99N999W,2005-09-08 16:03:37.927,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.66,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
3,MROX_0001,DATA/CRU_000004_9999_XN_99N999W.IMG,4A_04_0001000700,CRU_000004_9999_XN_99N999W,2005-09-08 16:08:23.841,CTX,NIFL,5056,2048,1,...,0.0,0.0,0.0,0.0,284.48,4.74,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126
4,MROX_0001,DATA/CRU_000005_9999_XN_99N999W.IMG,4A_04_0001000800,CRU_000005_9999_XN_99N999W,2005-09-08 16:11:18.649,CTX,NIFL,5056,21504,1,...,0.0,0.0,0.0,0.0,284.48,4.79,0.0,Calibration image of the Moon,OK,-4126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147009,MROX_4825,DATA/U23_080140_1972_XN_17N024W.IMG,4A_04_119602D100,U23_080140_1972_XN_17N024W,2023-08-31 20:06:02.042,CTX,NIFL,5056,14336,1,...,23.42,23.86,17.32,243001452.0,112.39,15.54,90.1,Ride-along with HiRISE,OK,80140
147010,MROX_4825,DATA/U23_080140_2621_XN_82N047W.IMG,4A_04_119602D200,U23_080140_2621_XN_82N047W,2023-08-31 20:26:46.409,CTX,NIFL,5056,7168,1,...,23.42,47.11,82.15,243000119.7,112.4,14.33,89.9,Ride-along with HiRISE,OK,80140
147011,MROX_4825,DATA/U23_080141_1718_XN_08S046W.IMG,4A_04_119602D300,U23_080141_1718_XN_08S046W,2023-08-31 21:50:19.874,CTX,NIFL,5056,12288,1,...,23.42,48.05,-8.43,242992745.9,112.43,15.71,90.0,Ride-along with HiRISE,OK,80141
147012,MROX_4825,DATA/U23_080141_2543_XN_74N064W.IMG,4A_04_119602D400,U23_080141_2543_XN_74N064W,2023-08-31 22:16:22.257,CTX,NIFL,4000,11264,1,...,23.42,64.19,74.38,242990081.8,112.44,14.96,89.9,Aspledon Undae,OK,80141


In [ ]:
#| export
def find_instruments(
        mission: str,  # Mission string, e.g. "cassini"
) -> list:  # List of configured instrument names
    "Find existing instruments for a mission."
    return config.list_instruments(mission)

In [ ]:
find_instruments("cassini")

['iss', 'uvis']

In [ ]:
find_instruments("mro")

['hirise', 'ctx']

In [ ]:
from nbdev import nbdev_export
nbdev_export()